In [1]:
# import essential libraries
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

In [2]:

# read in csv 
df = pd.read_csv('training_11_features.csv')
df = df.dropna()
# df = df[df['ritmi'] != 0]
df = df.reset_index(drop=True)
print(df.shape)
df.head()

(1803, 11)


,ritmi,age,sex,height,weight,heart_axis,validated_by,second_opinion,validated_by_human,pacemaker,strat_fold
0,2,29.0,1,164.0,56.0,0,0.0,0,1,0,1
1,0,59.0,0,156.0,75.0,0,0.0,0,1,0,9
2,2,84.0,1,152.0,51.0,0,0.0,0,1,0,7
3,0,79.0,0,172.0,66.0,0,0.0,0,1,0,5
4,1,67.0,0,178.0,73.0,4,0.0,0,1,0,5


In [3]:
# convert all columns' types to int64
df['age'] = df['age'].astype('int64')
df['height'] = df['height'].astype('int64')
df['weight'] = df['weight'].astype('int64')
df['validated_by'] = df['validated_by'].astype('int64')

# get info for columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1803 entries, 0 to 1802
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   ritmi               1803 non-null   int64
 1   age                 1803 non-null   int64
 2   sex                 1803 non-null   int64
 3   height              1803 non-null   int64
 4   weight              1803 non-null   int64
 5   heart_axis          1803 non-null   int64
 6   validated_by        1803 non-null   int64
 7   second_opinion      1803 non-null   int64
 8   validated_by_human  1803 non-null   int64
 9   pacemaker           1803 non-null   int64
 10  strat_fold          1803 non-null   int64
dtypes: int64(11)
memory usage: 155.1 KB


In [4]:
# train-test split
X = df.drop(columns = ['heart_axis','pacemaker','second_opinion','strat_fold','validated_by','validated_by_human','ritmi'])
y = df['ritmi']
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size = 0.25, random_state = 1234)

In [5]:
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (1352, 4)
y_train (1352,)
X_test (451, 4)
y_test (451,)


# Random Forest

In [6]:
# Plug in appropriate max_depth and random_state parameters
rf = RandomForestClassifier()
rf_param_grid = {'n_estimators': [600], 'criterion': ['entropy'], 'max_depth': [60]} #0.4615443314230772

rf_cv= GridSearchCV(rf,rf_param_grid,cv=7,n_jobs=-1)

rf_cv.fit(X_train,y_train)

print("Best Score:" + str(rf_cv.best_score_))
print("Best Parameters: " + str(rf_cv.best_params_))

Best Score:0.4548978610727449
Best Parameters: {'criterion': 'entropy', 'max_depth': 60, 'n_estimators': 600}


In [7]:
y_train_pred = rf_cv.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       461
           1       0.98      0.97      0.97       358
           2       0.98      0.98      0.98       533

    accuracy                           0.98      1352
   macro avg       0.98      0.98      0.98      1352
weighted avg       0.98      0.98      0.98      1352



In [8]:
y_train_pred = rf_cv.predict(X_train)
print(classification_report(y_train, y_train_pred))

y_pred = rf_cv.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       461
           1       0.98      0.97      0.97       358
           2       0.98      0.98      0.98       533

    accuracy                           0.98      1352
   macro avg       0.98      0.98      0.98      1352
weighted avg       0.98      0.98      0.98      1352

              precision    recall  f1-score   support

           0       0.41      0.38      0.40       151
           1       0.36      0.43      0.39       116
           2       0.42      0.39      0.40       184

    accuracy                           0.40       451
   macro avg       0.40      0.40      0.40       451
weighted avg       0.40      0.40      0.40       451



# K Neighbors Classifier

In [9]:
clfl2 = KNeighborsClassifier()
parameters = {'n_neighbors': [150], 'weights': ['distance'], 'metric': ['euclidean']} #0.4889326226595599

fitmodel = GridSearchCV(clfl2, param_grid=parameters, cv=5, refit=True, scoring="accuracy", n_jobs=-1, verbose=3)
fitmodel.fit(X_train, y_train)
print(fitmodel.best_estimator_, fitmodel.best_params_, fitmodel.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 5/5] END metric=euclidean, n_neighbors=150, weights=distance;, score=0.470 total time=   0.0s
[CV 4/5] END metric=euclidean, n_neighbors=150, weights=distance;, score=0.504 total time=   0.0s
[CV 3/5] END metric=euclidean, n_neighbors=150, weights=distance;, score=0.437 total time=   0.0s
[CV 2/5] END metric=euclidean, n_neighbors=150, weights=distance;, score=0.506 total time=   0.0s
[CV 1/5] END metric=euclidean, n_neighbors=150, weights=distance;, score=0.480 total time=   0.0s
KNeighborsClassifier(metric='euclidean', n_neighbors=150, weights='distance') {'metric': 'euclidean', 'n_neighbors': 150, 'weights': 'distance'} 0.47927019270192706


In [10]:
y_pred = fitmodel.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.47      0.42      0.44       151
           1       0.42      0.51      0.46       116
           2       0.48      0.46      0.47       184

    accuracy                           0.46       451
   macro avg       0.46      0.46      0.46       451
weighted avg       0.46      0.46      0.46       451



## Deep learning models data preparation

In [14]:
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout,MaxPooling1D,LSTM
from tensorflow.keras.models import Sequential
from keras.utils import to_categorical

2022-12-05 14:08:29.724986: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 14:08:30.037567: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-05 14:08:30.106394: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-05 14:08:30.106431: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [15]:
#Reshape train and test data to (n_samples, 187, 1), where each sample is of size (187, 1)
X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

In [16]:
y_test_old = y_test.copy()

In [17]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## 1-D Convolution

In [18]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-12-05 14:09:20.770033: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-05 14:09:20.770301: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-05 14:09:20.770322: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (TEO-Tabeed): /proc/driver/nvidia/version does not exist
2022-12-05 14:09:20.771063: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
# fit network
history_1d = model.fit(X_train, y_train, epochs=20)
# evaluate model

Epoch 1/20
43/43 [==============================] - 0s 2ms/step - loss: 7.1305 - accuracy: 0.3365
Epoch 2/20
43/43 [==============================] - 0s 1ms/step - loss: 3.3585 - accuracy: 0.3706
Epoch 3/20
43/43 [==============================] - 0s 1ms/step - loss: 2.3195 - accuracy: 0.3802
Epoch 4/20
43/43 [==============================] - 0s 1ms/step - loss: 1.9292 - accuracy: 0.3639
Epoch 5/20
43/43 [==============================] - 0s 1ms/step - loss: 1.7092 - accuracy: 0.3543
Epoch 6/20
43/43 [==============================] - 0s 1ms/step - loss: 1.4492 - accuracy: 0.3846
Epoch 7/20
43/43 [==============================] - 0s 1ms/step - loss: 1.4181 - accuracy: 0.3609
Epoch 8/20
43/43 [==============================] - 0s 1ms/step - loss: 1.2582 - accuracy: 0.3972
Epoch 9/20
43/43 [==============================] - 0s 1ms/step - loss: 1.2028 - accuracy: 0.3905
Epoch 10/20
43/43 [==============================] - 0s 1ms/step - loss: 1.2434 - accuracy: 0.3624
Epoch 11/20
43/43 [

In [20]:
_, accuracy = model.evaluate(X_test, y_test)

15/15 [==============================] - 0s 1ms/step - loss: 1.0946 - accuracy: 0.3703


In [21]:
pred_result = model.predict(X_test)
final_pred = [np.argmax(i) for i in pred_result]
print(classification_report(y_test_old,final_pred))

15/15 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.40      0.48      0.44       151
           1       0.34      0.76      0.47       116
           2       0.60      0.03      0.06       184

    accuracy                           0.37       451
   macro avg       0.45      0.42      0.32       451
weighted avg       0.47      0.37      0.29       451



## LSTM

In [22]:
lstm_model = Sequential()
lstm_model.add(LSTM(100, input_shape=(X_train.shape[1],1)))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(100, activation="relu"))
lstm_model.add(Dense(y_train.shape[1], activation="softmax"))
lstm_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 3)                 303       
                                                                 
Total params: 51,203
Trainable params: 51,203
Non-trainable params: 0
_________________________________________________________________


In [23]:
# fit network
history_lstm = lstm_model.fit(X_train, y_train, epochs=100)
# evaluate model

Epoch 1/100
43/43 [==============================] - 1s 3ms/step - loss: 1.1040 - accuracy: 0.3787
Epoch 2/100
43/43 [==============================] - 0s 3ms/step - loss: 1.0927 - accuracy: 0.3720
Epoch 3/100
43/43 [==============================] - 0s 4ms/step - loss: 1.0648 - accuracy: 0.3913
Epoch 4/100
43/43 [==============================] - 0s 3ms/step - loss: 1.0401 - accuracy: 0.4001
Epoch 5/100
43/43 [==============================] - 0s 3ms/step - loss: 1.0278 - accuracy: 0.4038
Epoch 6/100
43/43 [==============================] - 0s 2ms/step - loss: 1.0197 - accuracy: 0.4320
Epoch 7/100
43/43 [==============================] - 0s 2ms/step - loss: 1.0139 - accuracy: 0.4305
Epoch 8/100
43/43 [==============================] - 0s 3ms/step - loss: 1.0270 - accuracy: 0.4046
Epoch 9/100
43/43 [==============================] - 0s 3ms/step - loss: 1.0179 - accuracy: 0.4120
Epoch 10/100
43/43 [==============================] - 0s 3ms/step - loss: 1.0117 - accuracy: 0.4349
Epoch 11/

In [24]:
_, accuracy = lstm_model.evaluate(X_test, y_test)

15/15 [==============================] - 0s 1ms/step - loss: 1.0458 - accuracy: 0.4412


In [25]:
lstm_pred_result = lstm_model.predict(X_test)

15/15 [==============================] - 0s 1ms/step


In [26]:
final_pred_lstm = [np.argmax(i) for i in lstm_pred_result]

In [27]:
print(classification_report(y_test_old,final_pred_lstm))

              precision    recall  f1-score   support

           0       0.46      0.42      0.44       151
           1       0.38      0.74      0.50       116
           2       0.57      0.27      0.37       184

    accuracy                           0.44       451
   macro avg       0.47      0.48      0.44       451
weighted avg       0.48      0.44      0.43       451

